### Description:

- script for processing un-automated datasets to compare with the tool (such as car crashes and traffic)

In [1]:
import osmnx as ox
import pandas as pd
from shapely.geometry import Point, Polygon
import geopandas as gpd
import os

def filter_crashes_within_area(data_file, place_name, output_file):
    """
    Reads crash data and saves entries located within the bounding area defined by osmnx's graph_from_place.

    Parameters:
        data_file (str): Path to the CSV file containing crash data.
        place_name (str): The place name to create the bounding area (e.g., "Fredericksburg, VA, USA").
        output_file (str): Path to save the filtered data.

    Returns:
        None: Saves the filtered data to a new CSV file.
    """

      # 1. Load the graph and convert it into a GeoDataFrame (bounding area)
    print("Loading graph for the specified place...")
    graph = ox.graph_from_place(place_name, network_type="drive")
    gdf_boundary = ox.geocode_to_gdf(place_name)
    print(gdf_boundary)
    
    # 2. Read the crash dataset
    print("Reading crash dataset...")
    df = pd.read_csv(data_file)

    # Check if the required columns (LAT and LON) exist
    if 'LAT' not in df.columns or 'LON' not in df.columns:
        print("Error: LAT and LON columns are missing in the dataset.")
        return

    # 3. Convert crash data to GeoDataFrame using LAT and LON
    print("Converting crash data to GeoDataFrame...")
    df['geometry'] = df.apply(lambda row: Point(row['LON'], row['LAT']), axis=1)
    gdf_crashes = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

    # 4. Filter crashes that fall within the bounding area
    print("Filtering crashes within the area...")
    gdf_crashes_within = gdf_crashes[gdf_crashes.within(gdf_boundary.unary_union)]

    # 5. Save the filtered crashes to a new CSV file
    print(f"Saving filtered crashes to {output_file}...")
    gdf_crashes_within.drop(columns=['geometry'], inplace=True)  # Remove geometry column before saving
    gdf_crashes_within.to_csv(output_file, index=False)

    print("Process complete. Filtered data saved successfully.")



def main():
    
    place_name = "fredericksburg, VA, USA"
    simple_place_name = place_name.split(",")[0]

    os.makedirs(f"../data/{simple_place_name}", exist_ok=True)

    data_file = f"../data/{simple_place_name}/features (1).csv"
    output_file = f"../data/{simple_place_name}/car_crashes_{simple_place_name}.csv"
    filter_crashes_within_area(data_file, place_name, output_file)



main()

Loading graph for the specified place...
                                            geometry  bbox_west  bbox_south  \
0  POLYGON ((-77.53259 38.30853, -77.53184 38.308...  -77.53259   38.270151   

   bbox_east  bbox_north   place_id  osm_type   osm_id        lat       lon  \
0 -77.446793   38.326638  320417545  relation  1633328  38.303184 -77.46054   

      class            type  place_rank  importance addresstype  \
0  boundary  administrative          12    0.555167        city   

             name                             display_name  
0  Fredericksburg  Fredericksburg, Virginia, United States  
Reading crash dataset...


FileNotFoundError: [Errno 2] No such file or directory: 'data/fredericksburg/features (1).csv'

: 

: 

### Viewing Crashes

In [ ]:
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
def find_nearest_pano(crash_df, ball_tree, pano_df, threshold=25):
    """
    Finds the nearest panoramic point to each crash point within a given threshold.
    """
    crash_coords = np.radians(crash_df[['LAT', 'LON']].values)  # Convert to radians for latitude and longitude
    distances, indices = ball_tree.query(crash_coords, k=1)  # Query nearest neighbor

    # Convert distances from radians to meters (Earth's radius ~ 6371000 meters)
    distances_meters = distances[:, 0] * 6371000

    # Assign pano_id if within threshold
    crash_df['matched_pano_id'] = [
        pano_df.iloc[indices[i][0]]['pano_id'] if distances_meters[i] <= threshold else None
        for i in range(len(distances_meters))
    ]

    return crash_df

def get_car_crashes_with_panaramics():
    simple_name = "fredericksburg"
    base_directory = f"data/{simple_name}"

    # Load datasets
    panoramic_dataset = pd.read_csv(f"{base_directory}/{simple_name}_panoramic_data.csv")
    crash_dataset = pd.read_csv(f"{base_directory}/car_crashes_{simple_name}.csv")

    # Check the column names of the panoramic dataset
    print(panoramic_dataset.columns)  # Debug: Check actual column names

    # Assuming the columns are different, adjust based on the output.
    # If the actual names are 'lat' and 'long', modify the following line:
    pano_coords = np.radians(panoramic_dataset[['lat', 'long']].values)  # Adjust based on your column names

    # Build the BallTree
    ball_tree = BallTree(pano_coords, metric='haversine')
    # Match crash points to panoramic points
    result_df = find_nearest_pano(crash_dataset, ball_tree, panoramic_dataset, threshold=25)

    # Display the result for verification
    print(result_df.head())

    # Remove rows where matched_pano_id is None
    result_df = result_df.dropna(subset=['matched_pano_id'])

    # Save the result to a new CSV file
    result_df.to_csv(f"{base_directory}/car_crashes_with_panoramics_{simple_name}.csv", index=False)

# Call the function to execute
get_car_crashes_with_panaramics()


In [ ]:
from geopy.distance import geodesic
import math
import osmnx as ox
import networkx as nx
import numpy as np
from shapely.geometry import Point
import pandas as pd
from shapely.geometry import Point, LineString
import import_ipynb
import SunGlareDetection 
import ast
from datetime import datetime, timedelta, timezone
import folium


def convert_military_integer_to_time(military_int):
    # Ensure the military integer is a string with leading zeros if necessary
    military_time = str(military_int).zfill(4)

    # Extract hours and minutes
    hours = int(military_time[:len(military_time)-2]) if len(military_time) > 2 else 0
    minutes = int(military_time[len(military_time)-2:])

    # Return formatted time
    return hours,minutes

def calculate_sun_glare_for_crashes(location):
    simple_name = location.split(",")[0]
    base_directory = f"data/{simple_name}"
    output_crash_path = f"{base_directory}/car_crashes_with_sun_glare_{simple_name}.csv"
    graph = ox.graph_from_place(location, network_type="drive")

    # Load the crash data
    crash_data = pd.read_csv(f"{base_directory}/car_crashes_with_panoramics_{simple_name}.csv")
    panoramic_data = pd.read_csv(f"{base_directory}/{simple_name}_panoramic_data.csv")
    panoramic_data['segment_headings'] = panoramic_data['segment_headings'].apply(ast.literal_eval)

    car_crashes_with_sun_glare = []

    # iterate to find the sun glare for each crash
    for index, row in crash_data.iterrows():
        print(f"==={index}===")
        print(f"    matched_pano_id: {row['matched_pano_id']}")
        # Get the crash coordinates
        lat, long = row['LAT'], row['LON']
        car_crash_date = row['Crash Date']
        month, day, year = car_crash_date.split("/")
        hour, minutes = convert_military_integer_to_time(row['Crash Military Time'])
        print(f"    {int(month)}/{day}/{year} {hour}:{minutes}")
        date_time = datetime(int(year), int(month), int(day), int(hour), int(minutes), 0, tzinfo=timezone.utc)
        segmentation_dataset_path = f"{base_directory}/{simple_name}_panoramic_imgs/panoramic_segmentation_maps"
     
        matched_pano_id = row['matched_pano_id']
        panoramic_img_path = f"{base_directory}/{simple_name}_panoramic_imgs/panoramic_imgs/{matched_pano_id}.jpg"
        print(f"    panoramic_img_path: {panoramic_img_path}")
        panoramic_row = panoramic_data[matched_pano_id == panoramic_data['pano_id']].iloc[0]
        # print(f"{panoramic_row['segment_headings'][0]}")
        # print(f"    panoramic_row: {panoramic_row}")
        has_sun_glare = SunGlareDetection.check_if_any_sun_glare_at_panoramic_with_datetime(panoramic_row, date_time, segmentation_dataset_path, panoramic_img_path)
        print(f"    has_sun_glare: {has_sun_glare}")

        car_crashes_with_sun_glare.append({
            'date_time': date_time,
            'lat': lat,
            'long': long,
            'has_sun_glare': has_sun_glare
        })

    # save as csv
    car_crashes_with_sun_glare_df = pd.DataFrame(car_crashes_with_sun_glare)
    car_crashes_with_sun_glare_df.to_csv(output_crash_path, index=False)

    # create map of accidents with sun glare
    # Filter rows where has_sun_glare is True
    sun_glare_data = car_crashes_with_sun_glare_df[car_crashes_with_sun_glare_df['has_sun_glare'] == True]


    # Initialize the map with a default location (mean of latitudes and longitudes)
    m = folium.Map(location=[sun_glare_data['lat'].mean(), sun_glare_data['long'].mean()], zoom_start=12)

    # Add circles for locations with sun glare
    for _, row in sun_glare_data.iterrows():
        folium.Circle(
            location=[row['lat'], row['long']],
            radius=6,  # Radius in meters
            color='red',
            fill=True,
            fill_opacity=0.6,
            popup=f"{lat},{long}\nDate/Time: {row['date_time']}"
        ).add_to(m)
    
    # Save the map as an HTML file
    m.save(f"{base_directory}/car_crashes_with_sun_glare_map_{simple_name}.html")





def main():
    location = "fredericksburg, VA, USA"
    calculate_sun_glare_for_crashes(location)


main()
